# ESP32 CIFAR-10 Test

## Modules

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import serial
from tensorflow.keras.datasets import cifar10
import time

from sklearn.metrics import classification_report, confusion_matrix

## CIFAR-10 Dataset

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [7]:
x_train[0]

array([[[ 59,  62,  63],
        [ 43,  46,  45],
        [ 50,  48,  43],
        ...,
        [158, 132, 108],
        [152, 125, 102],
        [148, 124, 103]],

       [[ 16,  20,  20],
        [  0,   0,   0],
        [ 18,   8,   0],
        ...,
        [123,  88,  55],
        [119,  83,  50],
        [122,  87,  57]],

       [[ 25,  24,  21],
        [ 16,   7,   0],
        [ 49,  27,   8],
        ...,
        [118,  84,  50],
        [120,  84,  50],
        [109,  73,  42]],

       ...,

       [[208, 170,  96],
        [201, 153,  34],
        [198, 161,  26],
        ...,
        [160, 133,  70],
        [ 56,  31,   7],
        [ 53,  34,  20]],

       [[180, 139,  96],
        [173, 123,  42],
        [186, 144,  30],
        ...,
        [184, 148,  94],
        [ 97,  62,  34],
        [ 83,  53,  34]],

       [[177, 144, 116],
        [168, 129,  94],
        [179, 142,  87],
        ...,
        [216, 184, 140],
        [151, 118,  84],
        [123,  92,  72]]

## ESP32 MCU Procedures

In [76]:
def init_serial(port_name):
    # Configure and open serial port
    port = serial.Serial(
        port=port_name,
        baudrate=115200,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_NONE,
        stopbits=serial.STOPBITS_ONE,
        xonxoff=False,
        rtscts=False,
        dsrdtr=False
    )
    
    port.reset_input_buffer()
    port.reset_output_buffer()
    return port

In [70]:
def send_to_mcu(port, data):
    bytes_written = port.write(data)
    return bytes_written

In [71]:
def read_result_from_mcu(port):
    line = port.read_until()   # read a '\n' terminated line
    return line

In [73]:
def infer_with_MCU(raw_image):
    img = np.array(raw_image, dtype=np.uint8)
    bytes_sent = send_to_mcu(serial_port, img)
    
    response_str = read_result_from_mcu(serial_port)
    response_str = response_str.decode("utf-8")
    predictions = np.fromstring(response_str, dtype=np.float32, sep=',')
    return predictions[:-1]
    # return None

## Serial port setup

In [77]:
SERIAL_PORT_NAME = '/dev/ttyUSB0'
# SERIAL_PORT_NAME = '/dev/ttyUSB1'
# SERIAL_PORT_NAME = '/dev/ttyUSB2'
# SERIAL_PORT_NAME = '/dev/ttyUSB3'
# SERIAL_PORT_NAME = '/dev/ttyUSB4'
# SERIAL_PORT_NAME = '/dev/ttyUSB5'

# Configure and open serial port
serial_port = init_serial(SERIAL_PORT_NAME)

# Check which port was really used
print("Opened serial port : {0}".format(serial_port.name))

Opened serial port : /dev/ttyUSB0


## Predictions

In [78]:
i = 0
right_preds = 0
for raw_image in x_test[:1]:
    
    result_mcu = infer_with_MCU(raw_image) # discard the last element (NaN)
    
    # print(f"tfl esp32 image({i}): y_pred = {np.argmax(result_mcu)} | y_test = {y_test[i][0]} -> {np.argmax(result_mcu) == y_test[i][0]}")

    if np.argmax(result_mcu) == y_test[i][0]:
        right_preds += 1
    i += 1

print("General Accuracy: ", right_preds/i)

General Accuracy:  1.0


## Statistics